# LangChain 1.0 Guardrails: PII Protection for HR Systems

**Module:** Guardrails and PII Detection with Middleware

**What you'll learn:**
- 🔒 PII Detection & Protection Strategies
- 🚫 **Redact** Strategy - Complete removal with type markers
- 🎭 **Mask** Strategy - Partial obscuring for usability  
- #️⃣ **Hash** Strategy - Deterministic one-way transformation
- ⛔ **Block** Strategy - Hard stops for compliance
- 🏥 Real-world HR compliance scenarios

**HR Use Cases:**
- Employee data handling in chat logs
- GDPR/HIPAA compliance for sensitive information
- Audit trails without exposing PII
- Customer service transcripts sanitization
- Multi-tenant HR systems with data isolation

**Time:** 2-3 hours

---

## 📚 PII Protection Strategy Comparison

| Strategy | Description | Example | Best For |
|----------|-------------|---------|----------|
| `redact` | Replace with `[REDACTED_TYPE]` | `[REDACTED_EMAIL]` | Complete anonymization, audit logs |
| `mask` | Partially obscure (e.g., last 4 digits) | `****-****-****-1234` | User-facing systems, partial visibility |
| `hash` | Replace with deterministic hash | `a8f5f167...` | Tracking without exposure, analytics |
| `block` | Raise exception when detected | Error thrown | Zero-tolerance compliance, regulated data |

---

## Setup: Install Dependencies

In [ ]:
# Install LangChain 1.0 and required packages
!pip install --pre -U langchain langchain-openai langgraph
!pip install langgraph-checkpoint-sqlite

## Setup: Configuration and Imports

In [ ]:
# Configure API key
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# Core imports
from langchain.agents import create_agent, AgentState
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langgraph.checkpoint.memory import InMemorySaver
from typing import Annotated, Callable
from datetime import datetime
from functools import wraps
import json
import re
import hashlib

print("✅ Setup complete!")

## Setup: HR Employee Database with Sensitive PII

In [ ]:
# HR Employee Database with PII
EMPLOYEES = {
    "101": {
        "name": "Priya Sharma",
        "email": "priya.sharma@company.com",
        "phone": "+91-9876543210",
        "ssn": "123-45-6789",
        "credit_card": "4532-1234-5678-9012",
        "ip_address": "192.168.1.100",
        "department": "Engineering",
        "role": "Senior Developer",
        "salary": 120000,
        "address": "123 MG Road, Bangalore, Karnataka 560001",
        "emergency_contact": "rahul.sharma@gmail.com"
    },
    "102": {
        "name": "Rahul Verma",
        "email": "rahul.verma@company.com",
        "phone": "+91-9876543211",
        "ssn": "987-65-4321",
        "credit_card": "5412-3456-7890-1234",
        "ip_address": "192.168.1.101",
        "department": "Engineering",
        "role": "Engineering Manager",
        "salary": 180000,
        "address": "456 Whitefield Road, Bangalore, Karnataka 560066",
        "emergency_contact": "sneha.verma@yahoo.com"
    },
    "103": {
        "name": "Anjali Patel",
        "email": "anjali.patel@company.com",
        "phone": "+91-9876543212",
        "ssn": "456-78-9012",
        "credit_card": "3782-822463-10005",
        "ip_address": "192.168.1.102",
        "department": "HR",
        "role": "HR Director",
        "salary": 200000,
        "address": "789 Koramangala, Bangalore, Karnataka 560095",
        "emergency_contact": "rohan.patel@outlook.com"
    }
}

print("✅ HR Database loaded with PII-rich records")
print(f"Total employees: {len(EMPLOYEES)}")
print("\n⚠️  WARNING: This database contains sensitive PII for demonstration purposes")

## Setup: HR Tools

In [ ]:
# Define HR tools that return PII
@tool
def get_employee_details(employee_id: Annotated[str, "Employee ID"]) -> str:
    """Get complete employee details including contact information."""
    if employee_id in EMPLOYEES:
        emp = EMPLOYEES[employee_id]
        return f"""Employee Details:
Name: {emp['name']}
Email: {emp['email']}
Phone: {emp['phone']}
Department: {emp['department']}
Role: {emp['role']}
Address: {emp['address']}
Emergency Contact: {emp['emergency_contact']}"""
    return f"Employee {employee_id} not found"

@tool
def get_financial_info(employee_id: Annotated[str, "Employee ID"]) -> str:
    """Get financial information including salary and payment details."""
    if employee_id in EMPLOYEES:
        emp = EMPLOYEES[employee_id]
        return f"""Financial Information for {emp['name']}:
Annual Salary: ₹{emp['salary']:,}
Payment Card: {emp['credit_card']}
SSN: {emp['ssn']}"""
    return f"Employee {employee_id} not found"

@tool
def get_system_access(employee_id: Annotated[str, "Employee ID"]) -> str:
    """Get system access information."""
    if employee_id in EMPLOYEES:
        emp = EMPLOYEES[employee_id]
        return f"""System Access for {emp['name']}:
Email: {emp['email']}
Last Login IP: {emp['ip_address']}
VPN Connected: Yes"""
    return f"Employee {employee_id} not found"

print("✅ HR tools configured!")

---
# Part 1: Understanding PII Types

## Built-in PII Types in LangChain 1.0

| PII Type | Description | Example |
|----------|-------------|----------|
| `email` | Email addresses | user@domain.com |
| `credit_card` | Credit card numbers | 4532-1234-5678-9012 |
| `ip` | IP addresses | 192.168.1.100 |
| `mac_address` | MAC addresses | 00:1B:44:11:3A:B7 |
| `url` | URLs | https://example.com |
| `phone` | Phone numbers | +91-9876543210 |
| `ssn` | Social Security Numbers | 123-45-6789 |

---

## Lab 1.1: PII Detection Utility

In [ ]:
class PIIDetector:
    """Detect various types of PII in text."""
    
    PATTERNS = {
        "email": r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b',
        "credit_card": r'\b(?:\d{4}[- ]?){3}\d{4}\b',
        "ip": r'\b(?:[0-9]{1,3}\.){3}[0-9]{1,3}\b',
        "phone": r'\+?\d{1,3}[-.]?\(?\d{3,4}\)?[-.]?\d{3,4}[-.]?\d{4}',
        "ssn": r'\b\d{3}-\d{2}-\d{4}\b',
        "url": r'https?://(?:www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b(?:[-a-zA-Z0-9()@:%_\+.~#?&/=]*)',
    }
    
    @classmethod
    def detect(cls, text: str, pii_type: str) -> list:
        """Detect specific PII type in text."""
        pattern = cls.PATTERNS.get(pii_type)
        return re.findall(pattern, text) if pattern else []
    
    @classmethod
    def detect_all(cls, text: str) -> dict:
        """Detect all PII types in text."""
        results = {}
        for pii_type, pattern in cls.PATTERNS.items():
            matches = re.findall(pattern, text)
            if matches:
                results[pii_type] = matches
        return results

# Test
test_text = """Contact: priya.sharma@company.com, +91-9876543210
Card: 4532-1234-5678-9012, SSN: 123-45-6789, IP: 192.168.1.100"""

print("🔍 Testing PII Detection:\n")
print(f"Test Text: {test_text}\n")
print("Detected PII:")
for pii_type, values in PIIDetector.detect_all(test_text).items():
    print(f"  {pii_type}: {values}")

print("\n✅ PII Detector ready!")

---
# Part 2: REDACT Strategy 🚫

## Understanding Redaction

**Redaction** = Complete PII removal + Type marker

```
Input:  "Email: john@company.com, Card: 4532-1234-5678-9012"
Output: "Email: [REDACTED_EMAIL], Card: [REDACTED_CREDIT_CARD]"
```

**When to Use:**
- ✅ Audit logs for compliance
- ✅ Public data releases  
- ✅ Training ML models
- ✅ Complete anonymization needed

---

## Lab 2.1: Redact Middleware Implementation

In [ ]:
class RedactPIIGuardrail:
    """Guardrail to redact PII using @after_model decorator pattern."""
    
    def __init__(self, pii_types: list[str]):
        self.pii_types = pii_types
        self.redaction_log = []
    
    def redact_text(self, text: str) -> str:
        """Redact PII from text."""
        redacted = text
        
        for pii_type in self.pii_types:
            pattern = PIIDetector.PATTERNS.get(pii_type)
            if pattern:
                matches = re.findall(pattern, redacted)
                for match in matches:
                    replacement = f"[REDACTED_{pii_type.upper()}]"
                    redacted = redacted.replace(match, replacement)
                    self.redaction_log.append({
                        "type": pii_type,
                        "original": match,
                        "timestamp": datetime.now().isoformat()
                    })
        
        return redacted
    
    def __call__(self, state: AgentState) -> dict:
        """Process messages after model generates response."""
        messages = state.get("messages", [])
        if not messages:
            return {}
        
        # Get the last message (AI response)
        last_msg = messages[-1]
        
        if hasattr(last_msg, 'content') and last_msg.content:
            original = last_msg.content
            redacted = self.redact_text(original)
            
            if original != redacted:
                print(f"\n🚫 [REDACT] PII detected and redacted from response")
                
                # Create new message with redacted content
                new_msg = AIMessage(content=redacted)
                if hasattr(last_msg, 'tool_calls'):
                    new_msg.tool_calls = last_msg.tool_calls
                
                # Replace last message
                new_messages = messages[:-1] + [new_msg]
                return {"messages": new_messages}
        
        return {}
    
    def get_redaction_log(self):
        return self.redaction_log

print("✅ RedactPIIGuardrail created!")

## Lab 2.2: Test Redact Strategy

In [ ]:
# Create redaction guardrail
redact_guardrail = RedactPIIGuardrail(
    pii_types=["email", "phone", "ssn", "credit_card", "ip"]
)

# Create agent with redaction
redact_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_employee_details, get_financial_info],
    middleware=[redact_guardrail],  # Apply as middleware
    prompt="""You are an HR assistant. Provide employee information when requested.
    Note: All PII is automatically redacted for compliance."""
)

print("=" * 70)
print("SCENARIO 1: Employee Details with REDACTION")
print("=" * 70)

result = redact_agent.invoke({
    "messages": [{"role": "user", "content": "Get all details for employee 101"}]
})

print("\n🤖 Agent Response (PII Redacted):")
print(result['messages'][-1].content)

# Show audit log
print("\n" + "=" * 70)
print("📋 REDACTION AUDIT LOG")
print("=" * 70)

for i, entry in enumerate(redact_guardrail.get_redaction_log(), 1):
    print(f"\n{i}. Type: {entry['type'].upper()}")
    print(f"   Original: {entry['original']}")
    print(f"   Time: {entry['timestamp']}")

print(f"\n✅ Total PII items redacted: {len(redact_guardrail.get_redaction_log())}")

---
# Part 3: MASK Strategy 🎭

## Understanding Masking

**Masking** = Partial obscuring + Last few chars visible

```
Credit Card: 4532-1234-5678-9012 → ****-****-****-9012
Email: john.doe@company.com → j***@company.com  
Phone: +91-9876543210 → +91-****-**3210
SSN: 123-45-6789 → ***-**-6789
```

**When to Use:**
- ✅ Customer service interfaces
- ✅ Payment confirmations
- ✅ User verification
- ✅ Better UX than full redaction

---

## Lab 3.1: Mask Middleware Implementation

In [ ]:
class MaskPIIGuardrail:
    """Guardrail to mask PII."""
    
    MASK_STRATEGIES = {
        "email": lambda x: f"{x[0]}***@{x.split('@')[1]}" if '@' in x else x,
        "credit_card": lambda x: f"****-****-****-{x[-4:]}",
        "phone": lambda x: f"{x[:3]}-****-**{x[-4:]}",
        "ssn": lambda x: f"***-**-{x[-4:]}",
        "ip": lambda x: f"{'.'.join(x.split('.')[:2])}.***.**",
    }
    
    def __init__(self, pii_types: list[str]):
        self.pii_types = pii_types
        self.masking_log = []
    
    def mask_text(self, text: str) -> str:
        """Mask PII from text."""
        masked = text
        
        for pii_type in self.pii_types:
            pattern = PIIDetector.PATTERNS.get(pii_type)
            mask_func = self.MASK_STRATEGIES.get(pii_type)
            
            if pattern and mask_func:
                matches = re.findall(pattern, masked)
                for match in matches:
                    masked_value = mask_func(match)
                    masked = masked.replace(match, masked_value)
                    self.masking_log.append({
                        "type": pii_type,
                        "original": match,
                        "masked": masked_value,
                        "timestamp": datetime.now().isoformat()
                    })
        
        return masked
    
    def __call__(self, state: AgentState) -> dict:
        """Mask PII after model response."""
        messages = state.get("messages", [])
        if not messages:
            return {}
        
        last_msg = messages[-1]
        
        if hasattr(last_msg, 'content') and last_msg.content:
            original = last_msg.content
            masked = self.mask_text(original)
            
            if original != masked:
                print(f"\n🎭 [MASK] PII partially obscured in response")
                
                new_msg = AIMessage(content=masked)
                if hasattr(last_msg, 'tool_calls'):
                    new_msg.tool_calls = last_msg.tool_calls
                
                new_messages = messages[:-1] + [new_msg]
                return {"messages": new_messages}
        
        return {}
    
    def get_masking_log(self):
        return self.masking_log

print("✅ MaskPIIGuardrail created!")

## Lab 3.2: Test Mask Strategy

In [ ]:
# Create masking guardrail
mask_guardrail = MaskPIIGuardrail(
    pii_types=["email", "phone", "ssn", "credit_card", "ip"]
)

# Create agent with masking
mask_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_employee_details, get_financial_info],
    middleware=[mask_guardrail],
    prompt="""You are an HR assistant. Provide employee information.
    Note: Sensitive PII is partially masked for security."""
)

print("=" * 70)
print("SCENARIO 2: Financial Info with MASKING")
print("=" * 70)

result = mask_agent.invoke({
    "messages": [{"role": "user", "content": "Get financial info for employee 102"}]
})

print("\n🤖 Agent Response (PII Masked):")
print(result['messages'][-1].content)

# Show masking log
print("\n" + "=" * 70)
print("📋 MASKING AUDIT LOG")
print("=" * 70)

for i, entry in enumerate(mask_guardrail.get_masking_log(), 1):
    print(f"\n{i}. Type: {entry['type'].upper()}")
    print(f"   Original: {entry['original']}")
    print(f"   Masked:   {entry['masked']}")

print(f"\n✅ Total PII items masked: {len(mask_guardrail.get_masking_log())}")

---
# Part 4: HASH Strategy #️⃣

## Understanding Hashing

**Hashing** = One-way deterministic transformation

```
Email: john@company.com → [HASH_EMAIL_a8f5f167f44f]
Same email always produces same hash
Cannot reverse hash to get original
```

**When to Use:**
- ✅ Analytics without exposing PII
- ✅ User tracking across sessions
- ✅ De-duplication
- ✅ Data joins without identity

---

## Lab 4.1: Hash Middleware Implementation

In [ ]:
class HashPIIGuardrail:
    """Guardrail to hash PII."""
    
    def __init__(self, pii_types: list[str], salt: str = "hr_salt_2024", hash_length: int = 12):
        self.pii_types = pii_types
        self.salt = salt
        self.hash_length = hash_length
        self.hashing_log = []
        self.hash_mapping = {}  # For reference/debugging
    
    def hash_value(self, value: str, pii_type: str) -> str:
        """Create deterministic hash."""
        salted = f"{self.salt}_{value}"
        hash_hex = hashlib.sha256(salted.encode()).hexdigest()[:self.hash_length]
        return f"[HASH_{pii_type.upper()}_{hash_hex}]"
    
    def hash_text(self, text: str) -> str:
        """Hash PII from text."""
        hashed = text
        
        for pii_type in self.pii_types:
            pattern = PIIDetector.PATTERNS.get(pii_type)
            if pattern:
                matches = re.findall(pattern, hashed)
                for match in matches:
                    hashed_value = self.hash_value(match, pii_type)
                    hashed = hashed.replace(match, hashed_value)
                    
                    self.hash_mapping[hashed_value] = match
                    self.hashing_log.append({
                        "type": pii_type,
                        "original": match,
                        "hash": hashed_value,
                        "timestamp": datetime.now().isoformat()
                    })
        
        return hashed
    
    def __call__(self, state: AgentState) -> dict:
        """Hash PII after model response."""
        messages = state.get("messages", [])
        if not messages:
            return {}
        
        last_msg = messages[-1]
        
        if hasattr(last_msg, 'content') and last_msg.content:
            original = last_msg.content
            hashed = self.hash_text(original)
            
            if original != hashed:
                print(f"\n#️⃣ [HASH] PII replaced with deterministic hashes")
                
                new_msg = AIMessage(content=hashed)
                if hasattr(last_msg, 'tool_calls'):
                    new_msg.tool_calls = last_msg.tool_calls
                
                new_messages = messages[:-1] + [new_msg]
                return {"messages": new_messages}
        
        return {}
    
    def get_hashing_log(self):
        return self.hashing_log

print("✅ HashPIIGuardrail created!")

## Lab 4.2: Test Hash Strategy

In [ ]:
# Create hashing guardrail
hash_guardrail = HashPIIGuardrail(
    pii_types=["email", "phone", "ssn", "credit_card"],
    salt="secure_hr_salt_2024",
    hash_length=12
)

# Create agent
hash_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_employee_details, get_financial_info],
    middleware=[hash_guardrail],
    prompt="""You are an HR analytics assistant. 
    PII is hashed for analytics while maintaining referential integrity."""
)

print("=" * 70)
print("SCENARIO 3: Employee Data for Analytics with HASHING")
print("=" * 70)

result = hash_agent.invoke({
    "messages": [{"role": "user", "content": "Get contact details for employee 103"}]
})

print("\n🤖 Agent Response (PII Hashed):")
print(result['messages'][-1].content)

# Show hashing log
print("\n" + "=" * 70)
print("📋 HASHING AUDIT LOG")
print("=" * 70)

for i, entry in enumerate(hash_guardrail.get_hashing_log(), 1):
    print(f"\n{i}. Type: {entry['type'].upper()}")
    print(f"   Original: {entry['original']}")
    print(f"   Hash:     {entry['hash']}")

# Demonstrate deterministic property
print("\n" + "=" * 70)
print("🔄 DEMONSTRATING DETERMINISTIC HASHING")
print("=" * 70)

result2 = hash_agent.invoke({
    "messages": [{"role": "user", "content": "Show me employee 103 info again"}]
})

print("\n✅ Same PII = Same Hash (deterministic):")
print(f"Total hashes: {len(hash_guardrail.get_hashing_log())}")

---
# Part 5: BLOCK Strategy ⛔

## Understanding Blocking

**Blocking** = Raise exception when PII detected

```
Input: "My SSN is 123-45-6789"
Result: PIIDetectionError - Operation blocked!
```

**When to Use:**
- ✅ Zero-tolerance compliance
- ✅ Healthcare (HIPAA)
- ✅ Finance (PCI-DSS)
- ✅ Systems that log all data
- ✅ Third-party integrations

---

## Lab 5.1: Block Middleware Implementation

In [ ]:
class PIIDetectionError(Exception):
    """Raised when PII is detected in block mode."""
    pass

class BlockPIIGuardrail:
    """Guardrail to block operations when PII detected."""
    
    def __init__(self, pii_types: list[str], check_input: bool = True, check_output: bool = False):
        self.pii_types = pii_types
        self.check_input = check_input
        self.check_output = check_output
        self.detection_log = []
    
    def detect_pii(self, text: str) -> dict:
        """Detect PII in text."""
        detections = {}
        for pii_type in self.pii_types:
            pattern = PIIDetector.PATTERNS.get(pii_type)
            if pattern:
                matches = re.findall(pattern, text)
                if matches:
                    detections[pii_type] = matches
        return detections
    
    def __call__(self, state: AgentState) -> dict:
        """Check for PII and block if found."""
        messages = state.get("messages", [])
        if not messages:
            return {}
        
        # Check input (user message)
        if self.check_input:
            for msg in messages:
                if hasattr(msg, 'type') and msg.type == "human" and hasattr(msg, 'content'):
                    detections = self.detect_pii(msg.content)
                    if detections:
                        self.detection_log.append({
                            "location": "input",
                            "detections": detections,
                            "timestamp": datetime.now().isoformat()
                        })
                        
                        error_msg = f"""⛔ PII DETECTED - Operation Blocked!

Detected PII types: {', '.join(detections.keys())}

This system has zero-tolerance for PII in user input for compliance reasons.
Please remove sensitive information and try again."""
                        
                        print(f"\n{error_msg}")
                        
                        # Stop execution
                        return {
                            "messages": [("assistant", error_msg)],
                            "jump_to": "__end__"
                        }
        
        # Check output (AI response)
        if self.check_output:
            last_msg = messages[-1]
            if hasattr(last_msg, 'content') and last_msg.content:
                detections = self.detect_pii(last_msg.content)
                if detections:
                    self.detection_log.append({
                        "location": "output",
                        "detections": detections,
                        "timestamp": datetime.now().isoformat()
                    })
                    
                    error_msg = f"""⛔ CRITICAL: PII in Model Output!

Detected: {', '.join(detections.keys())}

This response has been blocked for compliance."""
                    
                    print(f"\n{error_msg}")
                    
                    return {
                        "messages": messages[:-1] + [("assistant", error_msg)],
                        "jump_to": "__end__"
                    }
        
        return {}
    
    def get_detection_log(self):
        return self.detection_log

print("✅ BlockPIIGuardrail created!")

## Lab 5.2: Test Block Strategy

In [ ]:
# Create blocking guardrail
block_guardrail = BlockPIIGuardrail(
    pii_types=["email", "ssn", "credit_card"],
    check_input=True,
    check_output=False  # Set to True to also block output
)

# Create agent
block_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_employee_details],
    middleware=[block_guardrail],
    prompt="""You are an HR assistant with strict PII protection.
    Any PII in user input will be blocked."""
)

print("=" * 70)
print("SCENARIO 4A: Clean Query (No PII) - ALLOWED")
print("=" * 70)

result = block_agent.invoke({
    "messages": [{"role": "user", "content": "Tell me about employee 101's department"}]
})

print("\n✅ Query allowed (no PII detected)")
print(f"Response: {result['messages'][-1].content[:100]}...")

print("\n" + "=" * 70)
print("SCENARIO 4B: Query with PII - BLOCKED")
print("=" * 70)

result = block_agent.invoke({
    "messages": [{"role": "user", "content": "My email is john@example.com and SSN is 123-45-6789"}]
})

print(f"\nResponse: {result['messages'][-1].content}")

# Show detection log
print("\n" + "=" * 70)
print("📋 BLOCK DETECTION LOG")
print("=" * 70)

for i, entry in enumerate(block_guardrail.get_detection_log(), 1):
    print(f"\n{i}. Location: {entry['location'].upper()}")
    print(f"   Detected Types: {list(entry['detections'].keys())}")
    print(f"   Time: {entry['timestamp']}")

print(f"\n⛔ Total blocked attempts: {len(block_guardrail.get_detection_log())}")

---
# Part 6: Combining Multiple Guardrails

In production, you might want **layered protection**:
- Block SSN/Credit Cards (zero-tolerance)
- Mask emails and phones (usability)
- Hash for analytics

---

## Lab 6.1: Multi-Layer PII Protection

In [ ]:
# Create layered guardrails
block_critical = BlockPIIGuardrail(
    pii_types=["ssn", "credit_card"],  # Zero tolerance
    check_input=True,
    check_output=True
)

mask_moderate = MaskPIIGuardrail(
    pii_types=["email", "phone"]  # Partial masking
)

# Apply both
production_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_employee_details, get_financial_info],
    middleware=[block_critical, mask_moderate],  # Order matters!
    prompt="""You are an HR assistant with multi-layer PII protection:
    - SSN/Credit Cards: BLOCKED (zero-tolerance)
    - Email/Phone: MASKED (partial visibility)"""
)

print("=" * 70)
print("PRODUCTION SCENARIO: Multi-Layer Protection")
print("=" * 70)

# Test 1: Query that should work with masking
print("\nTest 1: Employee details (emails will be masked)")
result = production_agent.invoke({
    "messages": [{"role": "user", "content": "Get contact info for employee 101"}]
})
print(f"✅ Response: {result['messages'][-1].content[:150]}...")

# Test 2: Query with critical PII (should be blocked)
print("\n" + "="*70)
print("Test 2: Financial info (SSN/Card will be blocked)")
result = production_agent.invoke({
    "messages": [{"role": "user", "content": "Get financial info for employee 102"}]
})
print(f"Response: {result['messages'][-1].content}")

print("\n" + "="*70)
print("📊 FINAL STATISTICS")
print("="*70)
print(f"Blocked attempts: {len(block_critical.get_detection_log())}")
print(f"Masked items: {len(mask_moderate.get_masking_log())}")

print("\n✅ Multi-layer PII protection demonstrated!")

---
# Summary & Best Practices

## Strategy Selection Guide

| Scenario | Recommended Strategy | Rationale |
|----------|---------------------|------------|
| **Audit logs** | REDACT | Complete anonymization required |
| **Customer service UI** | MASK | Balance security & usability |
| **Analytics pipeline** | HASH | Track without identity exposure |
| **HIPAA/PCI systems** | BLOCK | Zero-tolerance compliance |
| **Production HR** | BLOCK + MASK | Layered protection |

## Key Takeaways

✅ **Redact**: Complete removal, GDPR compliant, audit-friendly  
✅ **Mask**: Partial visibility, better UX, verification-friendly  
✅ **Hash**: Analytics-safe, deterministic, tracking-enabled  
✅ **Block**: Zero-tolerance, compliance-first, error-raising

## Production Checklist

- [ ] Identify all PII types in your system
- [ ] Map compliance requirements to strategies
- [ ] Implement layered protection (block + mask/redact)
- [ ] Log all PII detections for audit
- [ ] Test edge cases (malformed PII, encoding)
- [ ] Document strategy choices for compliance
- [ ] Regular review of detection patterns
- [ ] Train team on PII handling policies

## Next Steps

1. **Custom PII Types**: Add organization-specific patterns
2. **Advanced Detection**: ML-based PII detection
3. **Contextual Protection**: Different rules per user role
4. **Audit Dashboard**: Visualize PII protection metrics

---

**Congratulations!** You now understand all 4 PII protection strategies and can implement compliant HR systems! 🎉